In [29]:
## Importing Libraries ##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import xlrd
from sklearn import svm
import openpyxl
from sklearn.linear_model import TweedieRegressor
from timeit import default_timer as timer
from statsmodels.tsa.arima_model import ARMA
from datetime import datetime
from sklearn import *
import copy
import numpy as np
from numpy import nan
from scipy import stats
import numpy as np

In [30]:
## Declaring variable ##
df = pd.read_csv('accumulation_dataset.csv')
full_merged = pd.DataFrame()

In [31]:

for i in df.LOC_NAME.unique():

 ## Processing data ## 
    test = df[df.LOC_NAME == str(i)]
    test['change'] = test.PRCP.diff()
    test = test.dropna()
    test['z'] = np.abs(stats.zscore(test['change']))
    
    ## Seperating the guages measurements - Extremely frustering, please reexamine this ##
    testTwo = test['PRCP'].mask((test['PRCP'].shift(1) > test['PRCP']) & 
          (test['PRCP'].shift(1) > test['PRCP'].shift(-1)) & 
          (test['PRCP'].shift(5) > test['PRCP'].shift(-2)) & 
          (test['PRCP'].shift(4) > test['PRCP'].shift(-3)) &
          (test['PRCP'].shift(3) > test['PRCP'].shift(-4)) &
          (test['PRCP'].shift(5) > test['PRCP'].shift(-5)) &
          (test['PRCP'].shift(2) > test['PRCP'].shift(-6))).isna() 
    
    
    ## Idenitfying measurements by measurementID ##
    counter, count, bin, binvalues = 0, [], 1, []
    for i in testTwo:
        if i == False:
            counter = counter + 1; count.append(counter); binvalues.append(bin)
        if i == True:
            bin = bin + 1
            counter = 1; count.append(counter); binvalues.append(bin)   
    test['counter'], test['measurementID'] = count, binvalues 
    
    
    ## Merging all measurement_ID together ##
    df_merged = pd.DataFrame()
    for p in test.measurementID.unique():
            tes = test[test['measurementID'] == int(p)]
            tes.PRCP = tes.PRCP.rolling(10).median()
            tes.PRCP.values[0:10] = test[test['measurementID'] == int(p)]['PRCP'][0:10]
            tes.PRCP = tes.where(((tes['change'] >= 0.0).shift(-1)) & ((tes['change'] >= 0.0)) & ((tes['change'] >= 0.0).shift(1)) , np.nan)['PRCP'] 
            df_merged = pd.concat([df_merged,tes])
            
    ## Merging all locations into one ##
    full_merged = pd.concat([full_merged,df_merged.rename(columns={'0': str(i)})])
  

C:\Users\matty enright\anaconda3\envs\NewEnvironment\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\matty enright\anaconda3\envs\NewEnvironment\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [32]:
## Final Processing ##
full_merged['change'] = full_merged.PRCP.diff()
full_merged = full_merged.dropna()
full_merged['z'] = np.abs(stats.zscore(full_merged['change']))

In [33]:
## Declaring variable ##
df = full_merged
full_merged = pd.DataFrame()

In [34]:
## Changing values to date time ##
df.OBS_TIME_LOC = pd.to_datetime(df.OBS_TIME_LOC)

## creating a pivot table 
table = pd.pivot_table(df, values='PRCP', index=['OBS_TIME_LOC'], columns=['LOC_NAME'])

## changing values to days ##
table = table.resample('D').median()

## Setting the difference ##
table = table.diff()

## Removing all outliers ##
table.mask(table < 0, 2)

LOC_NAME,Burringbar,Burringbar North Arm (Harwood Rd),Burringbar Rd (Burringbar Creek),Byron Bay (Belongil Ck Bridge),Byron Bay (Belongil Creek),Clarrie Hall Dam (Doon Doon Creek),Coopers Shoot Repeater,Cudgera Creek (Pottsville Reservoir),Cudgera Lake,Doon Doon (Mccabes Road),...,Palmers Road,Repentance (Coopers Creek),Terania Creek,The Channon,Uki (Tweed River),Upper Burringbar Rd,Upper Crabbes Creek (Crabbes Creek Rd),Wooyung Rd (Crabbes Creek),Yelgun (Yelgun Creek),Yelgun Creek (Helen St Bridge)
OBS_TIME_LOC,,,,,,,,,,,,,,,,,,,,,
2005-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-11-27,NaN,NaN,NaN,NaN,5.75,7.5,NaN,NaN,2.5,NaN,...,NaN,1.00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2005-11-28,NaN,NaN,NaN,NaN,32.25,29.5,NaN,NaN,7.5,5.5,...,NaN,10.25,NaN,6.0,12.0,NaN,NaN,NaN,NaN,NaN
2005-11-29,NaN,NaN,NaN,NaN,1.00,2.5,NaN,NaN,14.0,6.0,...,NaN,4.75,NaN,3.5,0.5,NaN,NaN,NaN,NaN,NaN
2005-11-30,NaN,NaN,NaN,NaN,0.00,1.5,NaN,NaN,5.0,0.0,...,NaN,0.00,NaN,0.5,0.5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-27,0.0,0.0,NaN,0.000,NaN,1.0,0.0,0.0,NaN,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-07-28,0.0,0.0,NaN,0.000,NaN,0.0,0.0,0.0,NaN,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2022-07-29,0.0,0.0,NaN,0.000,NaN,0.0,0.0,0.0,NaN,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [35]:
## Removing all Na values ##
table = table.drop(['Yelgun Creek (Helen St Bridge)','Upper Burringbar Rd',
                  'Burringbar North Arm (Harwood Rd)','Burringbar Rd (Burringbar Creek)',
                 'Byron Bay (Belongil Ck Bridge)','Coopers Shoot Repeater','Huonbrook','Yelgun (Yelgun Creek)','Wooyung Rd (Crabbes Creek)','Terania Creek','Burringbar',
                  'Cudgera Creek (Pottsville Reservoir)','Palmers Road','Main Arm','Cudgera Lake','Upper Crabbes Creek (Crabbes Creek Rd)'],axis=1)

table= table.dropna()

In [36]:
table

LOC_NAME,Byron Bay (Belongil Creek),Clarrie Hall Dam (Doon Doon Creek),Doon Doon (Mccabes Road),Dunoon,Goonengerry,Lacks Creek (Middle Pocket),Mullumbimby (Chincogan ),Mullumbimby (Upper Main Arm),Mullumbimby Creek (Mullumbimby Ck),Myocum,Repentance (Coopers Creek),The Channon,Uki (Tweed River)
OBS_TIME_LOC,,,,,,,,,,,,,
2005-11-28,32.25,29.5,5.50,6.0,15.5,18.0,2.00,25.0,17.5,28.75,10.25,6.0,12.00
2005-11-29,1.00,2.5,6.00,4.0,4.0,4.0,3.00,5.0,6.5,3.75,4.75,3.5,0.50
2005-11-30,0.00,1.5,0.00,2.0,0.0,0.0,1.00,1.0,1.5,4.00,0.00,0.5,0.50
2005-12-01,2.00,0.0,0.00,0.0,0.0,1.0,7.00,0.0,3.0,6.00,0.00,0.0,2.00
2005-12-02,14.25,1.0,3.25,2.5,0.5,28.0,7.25,4.0,1.5,1.00,0.00,5.0,6.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-11,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,-4.0,0.00,0.00,0.0,0.00
2019-08-12,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00
2019-08-13,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00


In [37]:
table.to_csv('cleaned_accumulation_dataset.csv')